In [ ]:
import json
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np
import csv
import openpyxl

csv = pd.read_csv('data.csv', encoding='cp949') #encoding='cp949'
address= csv['도로명 주소']
for i in range(len(address)):
    a = address[i].split(' ')
    address[i] = " ".join(a[:-1])

    #####Dataframe만들기######
address_df = pd.DataFrame({'주소': address, '공급 면적' : csv['공급 면적'], '매매 가격(만 원)' : csv['매매 가격(만 원)'], '평당 가격(만 원)' : csv['평당 가격(만 원)'], '전용 면적' : csv['전용 면적'], '전용률' : csv['전용률'], '매매 대상 층 수' : csv['매매 대상 층 수'], '건물 전체 층 수' : csv['건물 전체 층 수'], '방 개수' : csv['방 개수'], '욕실 개수' : csv['욕실 개수'], '총 주차 공간 개수' : csv['총 주차 공간 개수'], '세대 당 주차 공간 평균' : csv['세대 당 주차 공간 평균'], '취득세(만 원)' : csv['취득세(만 원)']})

# Excel 파일로 저장
address_df.to_excel("address_tmp.xlsx", index = False) # xlsx 파일로 변환

filename = "address_tmp.xlsx"
exelFile = openpyxl.load_workbook(filename)

sheet = exelFile.worksheets[0]

## 데이터 가져오기
rowCount = 1
for row in sheet.rows:	

	try:
		## geocoder 호출
		r = requests.get("http://apis.vworld.kr/new2coord.do?q="+row[0].value+"&output=json&epsg=epsg:4326&apiKey=FA2EBC4D-393C-3B17-9E93-98724D8DF8B7")

		## cell 설정 [ (10)J1 ~ J* : 위도 / (11)K1 ~ K* : 경도]
		lat_cell = sheet.cell(row = rowCount, column = 2)
		lng_cell = sheet.cell(row = rowCount, column = 3)

		## 데이터 추가
		lat_cell.value = r.json()["EPSG_4326_Y"]
		lng_cell.value = r.json()["EPSG_4326_X"]
	except KeyError as ke:
		lat_cell.value = 0
		lng_cell.value = 0
	except TypeError as te:
		lat_cell.value = 0
		lng_cell.value = 0
		
	rowCount = rowCount + 1

sheet['B1'] = '위도'
sheet['C1'] = '경도'

## 데이터 저장
exelFile.save("address.xlsx")
xlsx = pd.read_excel("address.xlsx")
xlsx.to_csv("data_apt.csv")

input_file_name = "data_apt.csv"
output_file_name = "data_anm.txt"
with open(input_file_name, "r", encoding="cp949", newline="") as input_file, open(output_file_name, "w", encoding="utf-8", newline="") as output_file:        
    reader = csv.reader(input_file)
    col_names = next(reader)

    for cols in reader:
        doc = {col_name: col for col_name, col in zip(col_names, cols)}
        print(json.dumps(doc, ensure_ascii=False), file=output_file)